# Sketch_to_terrain - train model



In [1]:
import os
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd '/content/drive/MyDrive/AI_Tech/Semestr1/Sketch_to_terrain_model'

/content/drive/MyDrive/AI_Tech/Semestr1/Sketch_to_terrain_model


In [3]:
!ls

outputs			      terrain_generator0.h5
terrain_discriminator0.h5     terrain_generator1000.h5
terrain_discriminator1000.h5  terrain_generator1500.h5
terrain_discriminator1500.h5  terrain_generator2000.h5
terrain_discriminator2000.h5  terrain_generator2500.h5
terrain_discriminator2500.h5  terrain_generator3000.h5
terrain_discriminator3000.h5  terrain_generator3500.h5
terrain_discriminator3500.h5  terrain_generator4000.h5
terrain_discriminator4000.h5  terrain_generator4500.h5
terrain_discriminator4500.h5  terrain_generator500.h5
terrain_discriminator500.h5   training_data.npz


## Library

In [4]:
# from model import *
import time
import numpy as np
import tensorflow as tf
from tensorflow.keras.optimizers import Adam, RMSprop
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
# from dataset_builder import *
from tensorflow_addons.layers import SpectralNormalization

from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.initializers import RandomNormal
import tensorflow.keras.backend as K

In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [3]:
tf.debugging.set_log_device_placement(True)

In [4]:
tf.config.list_physical_devices('GPU')

[]

In [5]:
from tensorflow.python.client import device_lib 
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 13919438478408031204
]


In [6]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


False

In [9]:
tf.__version__

'2.4.1'

## Code to train model

In [5]:
def UNet(shape):
    inputs = Input(shape)
    conv1 = SpectralNormalization(Conv2D(64, 3, activation='relu', padding='same'))(inputs)
    conv1 = SpectralNormalization(Conv2D(64, 3, activation='relu', padding='same'))(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = SpectralNormalization(Conv2D(128, 3, activation='relu', padding='same'))(pool1)
    conv2 = SpectralNormalization(Conv2D(128, 3, activation='relu', padding='same'))(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = SpectralNormalization(Conv2D(256, 3, activation='relu', padding='same'))(pool2)
    conv3 = SpectralNormalization(Conv2D(256, 3, activation='relu', padding='same'))(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = SpectralNormalization(Conv2D(512, 3, activation='relu', padding='same'))(pool3)
    conv4 = SpectralNormalization(Conv2D(512, 3, activation='relu', padding='same'))(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = SpectralNormalization(Conv2D(1024, 3, activation='relu', padding='same'))(pool4)
    conv5 = SpectralNormalization(Conv2D(1024, 3, activation='relu', padding='same'))(conv5)
    noise = Input((K.int_shape(conv5)[1], K.int_shape(conv5)[2], K.int_shape(conv5)[3]))
    conv5 = Concatenate()([conv5, noise])

    up6 = Conv2D(
        512,
        2,
        activation='relu',
        padding='same')(
        UpSampling2D(
            size=(
                2,
                2))(conv5))
    merge6 = Concatenate()([conv4, up6])
    conv6 = Conv2D(512, 3, activation='relu', padding='same')(merge6)
    conv6 = Conv2D(512, 3, activation='relu', padding='same')(conv6)

    up7 = Conv2D(
        256,
        2,
        activation='relu',
        padding='same')(
        UpSampling2D(
            size=(
                2,
                2))(conv6))
    merge7 = Concatenate()([conv3, up7])
    conv7 = Conv2D(256, 3, activation='relu', padding='same')(merge7)
    conv7 = Conv2D(256, 3, activation='relu', padding='same')(conv7)

    up8 = Conv2D(
        128,
        2,
        activation='relu',
        padding='same')(
        UpSampling2D(
            size=(
                2,
                2))(conv7))
    merge8 = Concatenate()([conv2, up8])
    conv8 = Conv2D(128, 3, activation='relu', padding='same')(merge8)
    conv8 = Conv2D(128, 3, activation='relu', padding='same')(conv8)

    up9 = Conv2D(
        64,
        2,
        activation='relu',
        padding='same')(
        UpSampling2D(
            size=(
                2,
                2))(conv8))
    up9 = ZeroPadding2D(((0, 1), (0, 1)))(up9)
    merge9 = Concatenate()([conv1, up9])
    conv9 = Conv2D(64, 3, activation='relu', padding='same')(merge9)
    conv9 = Conv2D(64, 3, activation='relu', padding='same')(conv9)
    conv9 = Conv2D(32, 3, activation='relu', padding='same')(conv9)
    conv10 = Conv2D(1, 1, activation='tanh')(conv9)

    model = Model([inputs, noise], conv10)
    model.summary()
    return model


def patch_discriminator(shape):
    init = RandomNormal(stddev=0.02)
    in_image = Input(shape=shape)
    cond_image = Input((225, 225, 4))
    conc_img = Concatenate()([in_image, cond_image])
    d = SpectralNormalization(Conv2D(64, (4, 4), strides=(2, 2), padding='same',
               kernel_initializer=init))(conc_img)
    d = LeakyReLU(alpha=0.2)(d)
    d = SpectralNormalization(Conv2D(128, (4, 4), strides=(2, 2),
               padding='same', kernel_initializer=init))(d)
    d = LeakyReLU(alpha=0.2)(d)
    d = SpectralNormalization(Conv2D(256, (4, 4), strides=(2, 2),
               padding='same', kernel_initializer=init))(d)
    d = LeakyReLU(alpha=0.2)(d)
    d = SpectralNormalization(Conv2D(512, (4, 4), strides=(2, 2),
               padding='same', kernel_initializer=init))(d)
    d = LeakyReLU(alpha=0.2)(d)
    d = SpectralNormalization(Conv2D(512, (4, 4), padding='same', kernel_initializer=init))(d)
    x = LeakyReLU(alpha=0.2)(d)
    output = SpectralNormalization(Conv2D(1, (4, 4), padding='same',
                    activation='sigmoid', kernel_initializer=init))(d)
    model = Model([in_image, cond_image], output)

    return model


def mount_discriminator_generator(g, d, image_shape):
    d.trainable = False
    input_gen = Input(shape=image_shape)
    input_noise = Input(shape=(14, 14, 1024))
    gen_out = g([input_gen, input_noise])
    output_d = d([gen_out, input_gen])
    model = Model(inputs=[input_gen, input_noise], outputs=[output_d, gen_out])
    model.summary()

    return model





def generate_real_samples(dataset, ground_trud_ds, n_samples, patch_size):
    ix = np.random.randint(0, dataset.shape[0], n_samples)
    X = dataset[ix]
    gt = ground_trud_ds[ix]
    y = np.ones((n_samples, patch_size, patch_size, 1))
    return X, y, gt


def generate_fake_samples(g_model, dataset, patch_size):
    w_noise = np.random.normal(0, 1, (dataset.shape[0], 14, 14, 1024))
    X = g_model.predict([dataset, w_noise])
    y = np.zeros((len(X), patch_size, patch_size, 1))
    return X, y


def sample_images(generator, source, target, idx):
    print(target.shape)
    target = np.uint8(target * 127.5 + 127.5)
    w_noise = np.random.normal(0, 1, (1, 14, 14, 1024))
    predicted = generator.predict([source, w_noise])
    im = np.uint8(predicted[0, ...] * 127.5 + 127.5)
    im_source = np.uint8(source[0, ...] * 255)
    print(im_source.shape)
    im_c = np.concatenate((np.squeeze(im, axis=-1), np.squeeze(target, axis=-1),
                           im_source[..., 0], im_source[..., 1], im_source[..., 2], im_source[..., 3]), axis=1)
    plt.imsave('./outputs/sketch_conversion' + str(idx) + '.png', im_c, cmap='terrain')


def test_gan():
    terrain_generator = load_model('terrain_generator26500.h5')
    data = np.load('training_data.npz')
    XTrain = data['x']
    YTrain = data['y']
    for i in range(200):
        source = XTrain[i:i + 1, ...]
        target = YTrain[i, ...]
        sample_images(terrain_generator, source, target, i)


def train_gan():
    data = np.load('training_data.npz')
    XTrain = data['x']
    YTrain = data['y']
    input_shape_gen = (XTrain.shape[1], XTrain.shape[2], XTrain.shape[3])
    input_shape_disc = (YTrain.shape[1], YTrain.shape[2], YTrain.shape[3])

    terrain_generator = UNet(input_shape_gen)
    terrain_discriminator = patch_discriminator(input_shape_disc)
    optd = Adam(0.0001, 0.5)
    terrain_discriminator.compile(loss='binary_crossentropy', optimizer=optd)
    composite_model = mount_discriminator_generator(
        terrain_generator, terrain_discriminator, input_shape_gen)
    composite_model.compile(
        loss=[
            'binary_crossentropy', 'mae'], loss_weights=[
            1, 3], optimizer=optd)

    n_epochs, n_batch, = 100, 20
    bat_per_epo = int(len(XTrain) / n_batch)
    patch_size = 15
    n_steps = bat_per_epo * n_epochs
    print('Bat_per_epo ',  bat_per_epo, "; n_steps: ", n_steps)
    min_loss = 999
    avg_loss = 0
    avg_dloss = 0
    for i in range(n_steps):
        print('n_steps: {}'.format(i)) # MM 19.04 - dodanie kontroli uczenia
        start_time = time.time() # MM 25.04 - dodatnie pobiarów czasowych w przeliczeniach step
        X_real, labels_real, Y_target = generate_real_samples(XTrain, YTrain, n_batch, patch_size)
        Y_target[np.isnan(Y_target)] = 0
        X_real[np.isnan(X_real)] = 0

        Y_fake, labels_fake = generate_fake_samples(terrain_generator, X_real, patch_size)
        w_noise = np.random.normal(0, 1, (n_batch, 14, 14, 1024))
        losses_composite = composite_model.train_on_batch(
            [X_real, w_noise], [labels_real, Y_target])

        loss_discriminator_fake = terrain_discriminator.train_on_batch(
            [Y_fake, X_real], labels_fake)
        loss_discriminator_real = terrain_discriminator.train_on_batch(
            [Y_target, X_real], labels_real)
        d_loss = (loss_discriminator_fake + loss_discriminator_real) / 2
        avg_dloss = avg_dloss + (d_loss - avg_dloss) / (i + 1)
        avg_loss = avg_loss + (losses_composite[0] - avg_loss) / (i + 1)
        end_time = time.time() # MM 25.04 - dodatnie pobiarów czasowych w przeliczeniach step
        print('total loss:' + str(avg_loss) + ' d_loss:' + str(avg_dloss) + ' time_calculate_step:'+str(end_time - start_time))

        if i % 100 == 0:
            sample_images(terrain_generator, X_real[0:1, ...], Y_target[0, ...], i)
        if i % 500 == 0:
            terrain_discriminator.save('terrain_discriminator' + str(i) + '.h5', True)
            terrain_generator.save('terrain_generator' + str(i) + '.h5', True)





In [ ]:
# extract_patches_from_raster()
# compute_sketches()
train_gan()
# test_gan()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 225, 225, 4) 0                                            
__________________________________________________________________________________________________
spectral_normalization (Spectra (None, 225, 225, 64) 2432        input_1[0][0]                    
__________________________________________________________________________________________________
spectral_normalization_1 (Spect (None, 225, 225, 64) 36992       spectral_normalization[0][0]     
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 112, 112, 64) 0           spectral_normalization_1[0][0]   
______________________________________________________________________________________________